In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map  

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Load the polygon
polygon = ee.Geometry.Polygon([[[-96.98455810546875, 27.754038315679356],[-97.2509765625, 27.82936085978979],[-97.39654541015625, 27.60323689456203],[-97.1026611328125, 27.520451064122117]]])
# Load the image from the archive.
image = ee.Image('LANDSAT/LC8_L1T/LC80260412017023LGN00')
# Define visualization parameters in an object literal.
vizParams = { 'bands': ['B5', 'B4', 'B3'], 'min': 5000, 'max': 15000, 'gamma': 1.3}

# Center the map on the image and display.
Map.centerObject(image, 7)
Map.addLayer(image, vizParams, 'Landsat 8 false color')

In [4]:
def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# 'B5_histogram'和 polygon、B5 要根据情景进行变更,
# Return the DN that maximizes interclass variance in B5 (in the region).

def otsu(image,band):
    histogram = image.select(band).reduceRegion(**{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': polygon, 
        'scale': 30,
        'bestEffort': True
    })
    counts = ee.Array(ee.Dictionary(histogram.get(band)).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram.get(band)).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
    indices = ee.List.sequence(1,size)
    # Compute between sum of squares, where each mean partitions the data.
    bss = indices.map(lambda i : sumSquare(i,histogram.get(band),counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

In [5]:
threshold = otsu(image,'B5')
print(threshold.getInfo())
# print('threshold : ', threshold.getInfo)

classA = image.select('B5').lt(threshold)
Map.addLayer(classA.updateMask(classA), {'palette': ['blue']}, 'class A')

8127.50055505975


In [ ]:
# # Compute the histogram of the NIR band.  The mean and variance are only FYI.
# histogram = image.select('B5').reduceRegion(**{
#     'reducer': ee.Reducer.histogram(255, 2),
#     'geometry': polygon, 
#     'scale': 30,
#     'bestEffort': True
# })
# print(histogram.getInfo())

# def sumSquare(i,histogram,counts,means,size,mean):
#     aCounts = counts.slice(0, 0, i)
#     aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
#     aMeans = means.slice(0, 0, i)
#     aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
#     bCounts = counts.slice(0, i, size)
#     bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
#     bMeans = means.slice(0, i, size)
#     bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

#     return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# # Return the DN that maximizes interclass variance in B5 (in the region).
# def otsu(histogram):
#     counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
#     means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
#     size = means.length().get([0])
#     mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
#     indices = ee.List.sequence(1,size)
#     # Compute between sum of squares, where each mean partitions the data.
#     bss = indices.map(lambda i : sumSquare(i,histogram,counts,means,size,mean))
#     # Return the mean value corresponding to the maximum BSS.
#     return means.sort(bss).get([-1])

# threshold = ee.Number(otsu(histogram.get('B5')))
# print(threshold.getInfo())
# # print('threshold : ', threshold.getInfo)